In [ ]:
import pandas as pd
import os
import glob
import duckdb
from strava_duck_funcs_copy import *
from fitparse import FitFile

ACTIVITIES_FILE = '/Users/bac/Documents/PYTHON_PROJECTS/Strava_Analysis/strava_data_dumps/STRAVA+export_8029714/activities.csv'


In [2]:

# --- MAIN FLOW ---

if __name__ == "__main__":
    create_tables()
    bulk_insert_activities(ACTIVITIES_FILE)
    #logs_df = build_all_fit_logs()
    #bulk_insert_fit_logs(logs_df)
    #test_query()
    print("\n🏁 All done!")


✅ Tables created fresh.
⚠️ activity_id already exists in DataFrame, skipping ID generation.

🏁 All done!


In [3]:
DB_FILE = '../outputs/strava_test_1.duckdb'
# Connect to DuckDB
con = duckdb.connect(DB_FILE)

In [6]:
def test_query():
    """Query activities joined with fit_logs."""
    df = con.execute("""
        SELECT distinct activity_type
        FROM activities
        -- JOIN fit_logs f ON a.activity_id = f.activity_id
        -- ORDER BY a.activity_id, f.timestamp
        LIMIT 10
    """).fetchdf()
    print("\n🎯 Sample joined data:")
    print(df)

In [7]:
test_query()


🎯 Sample joined data:
     ACTIVITY_TYPE
0             Ride
1             Walk
2              Run
3          Workout
4  Weight Training
5       Elliptical
6             Swim


# Basic DuckDB example -- Works

In [ ]:
import duckdb

# Connect to a DuckDB database file (it will create it if it doesn't exist)
con = duckdb.connect('test_project.duckdb')

# Drop tables if they exist (to start fresh)
con.execute("DROP TABLE IF EXISTS activities")
con.execute("DROP TABLE IF EXISTS fit_logs")

# Create activities table (MINIMAL, NO constraints)
con.execute("""
    CREATE TABLE activities (
        activity_id BIGINT,
        name TEXT,
        date TIMESTAMP,
        distance DOUBLE,
        duration DOUBLE,
        fit_filename TEXT
    )
""")

# Create fit_logs table (also MINIMAL)
con.execute("""
    CREATE TABLE fit_logs (
        log_id BIGINT,
        activity_id BIGINT,
        timestamp TIMESTAMP,
        field_name TEXT,
        field_value DOUBLE
    )
""")

print("✅ Tables created successfully.")

# Insert one fake record into activities
con.execute("""
    INSERT INTO activities (activity_id, name, date, distance, duration, fit_filename)
    VALUES (1, 'Morning Run', NOW(), 5.0, 30.0, 'run01.fit')
""")

# Insert one fake record into fit_logs
con.execute("""
    INSERT INTO fit_logs (log_id, activity_id, timestamp, field_name, field_value)
    VALUES (1, 1, NOW(), 'heart_rate', 145)
""")

print("✅ Inserted example data.")

# Query and print
df = con.execute("""
    SELECT a.name, f.timestamp, f.field_name, f.field_value
    FROM activities a
    JOIN fit_logs f ON a.activity_id = f.activity_id
""").fetchdf()

print("\n🎯 Query result:")
print(df)

con.close()


In [ ]:
df